# Notebook 03: Transition-State Theory
## **PROJECT: Design the Perfect Prodrug**

---

## PROJECT SCENARIO

You're a medicinal chemist at PharmaTech Solutions. Your team developed a potent anticancer drug, but it has a critical problem:

**The drug is TOO reactive! It degrades before reaching tumor cells.**

Solution: Design a **prodrug** - an inactive precursor that converts to the active drug *at the right rate*.

**Your challenge:**
- **Too slow** → No therapeutic effect (t½ > 24 hours)
- **Too fast** → Systemic toxicity (t½ < 30 minutes)
- **Just right** → Target: t½ = 2-4 hours at body pH and temperature

Your mission:
1. **Understand** the activation barrier using Transition-State Theory
2. **Measure** activation parameters (ΔH‡, ΔS‡) from experimental data
3. **Elucidate** the mechanism using Kinetic Isotope Effects
4. **Optimize** the molecular structure using Hammett analysis
5. **Recommend** the best prodrug candidate for clinical trials

---

## LEARNING OBJECTIVES

By the end of this project notebook, you will be able to:
- [ ] Apply the Eyring equation to calculate rate constants from activation energies
- [ ] Extract ΔH‡ and ΔS‡ from temperature-dependent kinetics data
- [ ] Use Kinetic Isotope Effects to determine reaction mechanisms
- [ ] Apply Hammett plots to predict structure-reactivity relationships
- [ ] Optimize reaction conditions (pH, ionic strength) using TST principles
- [ ] Design molecules with targeted activation barriers

**Self-Assessment**: Check off each objective as you complete it!

---

## PHASE 1: DISCOVER 🔍

Before diving into the data, test your understanding of transition-state theory.

### PRE-LAB QUESTIONS

**Question 1**: Your prodrug has ΔH‡ = 80 kJ/mol and ΔS‡ = -50 J/(K·mol). What does the negative entropy tell you?
- A) The reaction is very fast
- B) The transition state is more ordered than reactants
- C) The reaction requires a catalyst
- D) The reaction is endothermic

<details>
<summary><strong>Click to reveal answer</strong></summary>

**Answer: B) The transition state is more ordered than reactants**

Negative ΔS‡ means the system loses entropy (freedom) when forming the transition state. This typically happens in associative reactions where two molecules come together, or when the transition state has a tight, rigid geometry. This "order tax" makes the reaction slower than entropy-neutral reactions with the same ΔH‡.
</details>

**Question 2**: You measure kH/kD = 6.5 at 25°C for your prodrug activation. What does this tell you?
- A) Deuterium makes the drug more stable
- B) Hydrogen transfer occurs in the rate-limiting step
- C) The reaction involves quantum tunneling
- D) Both A and B

<details>
<summary><strong>Click to reveal answer</strong></summary>

**Answer: D) Both A and B**

A KIE of ~6-7 is close to the theoretical maximum for classical (non-tunneling) reactions and indicates:
1. C-H/C-D bond breaking is involved in the rate-limiting step
2. Replacing H with D raises the activation barrier (due to lower zero-point energy)
3. This can be used to stabilize prodrugs by deuteration!

If kH/kD >> 7, quantum tunneling would be suspected.
</details>

**Question 3**: Using Hammett analysis, you find that electron-withdrawing groups (EWG) on the benzene ring speed up prodrug activation (ρ > 0). What does this tell you about the transition state?
- A) The transition state has positive charge buildup
- B) The transition state has negative charge buildup
- C) The reaction is diffusion-controlled
- D) The reaction involves free radicals

<details>
<summary><strong>Click to reveal answer</strong></summary>

**Answer: B) The transition state has negative charge buildup**

If EWGs accelerate the reaction (ρ > 0), it means they stabilize the transition state. EWGs stabilize negative charge, so the transition state must be developing negative charge (e.g., nucleophilic attack). Conversely, ρ < 0 would indicate positive charge buildup (electrophilic attack).
</details>

---

### YOUR CHALLENGE

**Initial Screening Data**: Three prodrug candidates tested at 37°C, pH 7.4:
- **Candidate A**: t½ = 45 min (too fast - toxicity risk)
- **Candidate B**: t½ = 3.2 hours (within target range!)
- **Candidate C**: t½ = 18 hours (too slow - no efficacy)

**Make a prediction**:
- Which candidate has the highest activation barrier?
- How could you modify Candidate A to slow it down?
- How could you modify Candidate C to speed it up?

Write your hypothesis:
- My hypothesis: _______________________

---

## 1. Introduction: The Activated Complex

Collision theory is intuitive but hard to calculate (steric factors are empirical). **Transition-State Theory (TST)**, developed by Eyring, Evans, and Polanyi (1930s), takes a different approach. It focuses on the species at the top of the energy barrier: the **activated complex** or **transition state** ($C^\ddagger$).

### Key Assumptions
1.  **Quasi-Equilibrium**: Reactants A and B are in equilibrium with the activated complex $C^\ddagger$.
    $$ A + B \underset{k_{-1}}{\overset{k_1}{\rightleftharpoons}} C^\ddagger \xrightarrow{k^\ddagger} P $$
2.  **Motion along Reaction Coordinate**: The complex falls apart into products with a frequency $\nu^\ddagger$ (a "loose vibration").

![Reaction Coordinate](images/reaction_coordinate_diagram.png)

For prodrug design, this means we can:
- **Tune the barrier height** (ΔG‡) to control reaction rate
- **Engineer the transition state** structure to achieve desired kinetics
- **Predict how modifications** will affect activation rate

In [ ]:
# ============================================================
# GOOGLE COLAB SETUP
# ============================================================
import sys
import os

# Check if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("=" * 60)
    print("RUNNING IN GOOGLE COLAB")
    print("=" * 60)

    # Clone repository to access images
    repo_url = "https://github.com/mcbadlon31/Reaction-Dynamics-Physical-Chemistry.git"

    print(f"\nCloning repository: {repo_url}")
    print("This may take a minute...")

    !git clone {repo_url} --depth 1 --quiet

    # Change to repository directory
    os.chdir('Reaction-Dynamics-Physical-Chemistry')

    # Install additional packages if needed
    print("\nInstalling additional packages...")
    !pip install -q seaborn plotly ipywidgets

    print("\n" + "=" * 60)
    print("[SUCCESS] Colab setup complete!")
    print("=" * 60)
    print(f"Current directory: {os.getcwd()}")
    print("\nYou can now run all cells normally.")
    print("Images will load from the cloned repository.")

else:
    print("=" * 60)
    print("RUNNING IN LOCAL JUPYTER ENVIRONMENT")
    print("=" * 60)
    print("\nNo setup needed - using local files")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 120,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'grid.alpha': 0.3
})

k_B = constants.Boltzmann
h = constants.Planck
R = constants.gas_constant
N_A = constants.Avogadro

print("Libraries loaded.")

---

## PHASE 2: INVESTIGATE 🔬

Time to analyze your prodrug candidates and extract critical kinetic parameters!

You'll complete **three investigations**:

1. **Activation Parameter Extraction**: Measure ΔH‡ and ΔS‡ from Eyring plots
2. **Mechanism Elucidation**: Use KIE to identify the rate-limiting step
3. **Structure-Activity Optimization**: Apply Hammett analysis to design better candidates

Each investigation includes:
- Real experimental data from your lab
- Guided analysis and calculations
- Mechanistic interpretation

Let's solve the prodrug challenge!

## 2. The Eyring Equation

The concentration of the activated complex is given by statistical thermodynamics:
$$ K^\ddagger = \frac{[C^\ddagger]}{[A][B]} = \frac{q_{C^\ddagger}}{q_A q_B} e^{-E_0/RT} $$

**Crucial Step**: We factor out the vibrational mode corresponding to the reaction coordinate from the partition function $q_{C^\ddagger}$:
$$ q_{C^\ddagger} \approx \left(\frac{k_B T}{h \nu^\ddagger}\right) \bar{q}_{C^\ddagger} $$

The rate is $Rate = \nu^\ddagger [C^\ddagger]$. Substituting $[C^\ddagger]$:
$$ Rate = \nu^\ddagger \times \left( \frac{k_B T}{h \nu^\ddagger} \frac{\bar{q}_{C^\ddagger}}{q_A q_B} e^{-E_0/RT} \right) [A][B] $$

The frequency $\nu^\ddagger$ cancels out! This gives the **Eyring Equation**:
$$ k_r = \kappa \frac{k_B T}{h} \bar{K}^\ddagger $$
where $\kappa$ is the transmission coefficient (usually $\approx 1$).

In thermodynamic terms ($\Delta^\ddagger G^\circ = -RT \ln \bar{K}^\ddagger$):
$$ k_r = \kappa \frac{k_B T}{h} e^{-\Delta^\ddagger G^\circ / RT} $$

### Thermodynamic Formulation

We can break down the Gibbs energy of activation into enthalpy and entropy terms:
$$ \Delta^\ddagger G^\circ = \Delta^\ddagger H^\circ - T\Delta^\ddagger S^\circ $$

Substituting this into the Eyring equation:
$$ k_r = \frac{k_B T}{h} e^{\Delta^\ddagger S^\circ / R} e^{-\Delta^\ddagger H^\circ / RT} $$

Taking the logarithm:
$$ \ln\left(\frac{k}{T}\right) = \ln\left(\frac{k_B}{h}\right) + \frac{\Delta^\ddagger S^\circ}{R} - \frac{\Delta^\ddagger H^\circ}{R} \cdot \frac{1}{T} $$

**This is the Eyring Plot!** A plot of $\ln(k/T)$ vs. $1/T$ gives:
- **Slope** = $-\Delta^\ddagger H^\circ / R$ → Extract enthalpy of activation
- **Intercept** = $\ln(k_B/h) + \Delta^\ddagger S^\circ / R$ → Extract entropy of activation

---

## INVESTIGATION 1: Activation Parameter Extraction 📊

### YOUR TASK
Measure prodrug activation rates at different temperatures and extract ΔH‡ and ΔS‡ using Eyring analysis.

### EXERCISE 1.1: Eyring Plot Analysis

In [ ]:
# EXERCISE 1.1: Eyring Plot for Prodrug Activation

# Load temperature-dependent kinetics data
try:
    kinetics_data = pd.read_csv('data/tst/prodrug_kinetics_temperature.csv')
    print("✓ Temperature-dependent kinetics data loaded!")
    print(f"\nData shape: {kinetics_data.shape}")
    print(f"\nFirst few rows:")
    print(kinetics_data.head())
except FileNotFoundError:
    print("⚠️ Data file not found. Make sure you're in the repository directory.")
    kinetics_data = None

# YOUR TASK: Create Eyring and Arrhenius plots
if kinetics_data is not None:
    import scipy.stats as stats
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Prepare data for each candidate
    candidates = kinetics_data['candidate'].unique()
    colors = ['blue', 'green', 'red']
    
    results = {}
    
    for i, candidate in enumerate(candidates):
        data = kinetics_data[kinetics_data['candidate'] == candidate]
        
        T = data['temperature_K'].values
        k = data['rate_constant_s_inv'].values
        
        # Eyring plot: ln(k/T) vs 1/T
        x_eyring = 1/T
        y_eyring = np.log(k/T)
        
        # Linear regression
        slope_eyring, intercept_eyring, r_value, _, _ = stats.linregress(x_eyring, y_eyring)
        
        # Extract activation parameters
        dH_act = -slope_eyring * R  # J/mol
        dS_act = (intercept_eyring - np.log(k_B/h)) * R  # J/(K·mol)
        
        # Arrhenius plot: ln(k) vs 1/T for comparison
        y_arrhenius = np.log(k)
        slope_arrhenius, intercept_arrhenius, _, _, _ = stats.linregress(x_eyring, y_arrhenius)
        Ea = -slope_arrhenius * R
        
        results[candidate] = {
            'dH': dH_act / 1000,  # kJ/mol
            'dS': dS_act,  # J/(K·mol)
            'Ea': Ea / 1000,  # kJ/mol
            'r2': r_value**2
        }
        
        # Plot 1: Eyring plot
        axes[0].scatter(x_eyring*1000, y_eyring, s=100, alpha=0.7, 
                       c=colors[i], edgecolors='black', label=candidate)
        axes[0].plot(x_eyring*1000, slope_eyring*x_eyring + intercept_eyring, 
                    '--', color=colors[i], linewidth=2, alpha=0.7)
        
        # Plot 2: Arrhenius plot
        axes[1].scatter(x_eyring*1000, y_arrhenius, s=100, alpha=0.7,
                       c=colors[i], edgecolors='black', label=candidate)
        axes[1].plot(x_eyring*1000, slope_arrhenius*x_eyring + intercept_arrhenius,
                    '--', color=colors[i], linewidth=2, alpha=0.7)
    
    # Formatting
    axes[0].set_xlabel('1000 / T (K⁻¹)', fontsize=12)
    axes[0].set_ylabel('ln(k/T)', fontsize=12)
    axes[0].set_title('Eyring Plot', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].set_xlabel('1000 / T (K⁻¹)', fontsize=12)
    axes[1].set_ylabel('ln(k)', fontsize=12)
    axes[1].set_title('Arrhenius Plot (for comparison)', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print results table
    print("\n" + "="*80)
    print("ACTIVATION PARAMETERS FROM EYRING ANALYSIS")
    print("="*80)
    print(f"\n{'Candidate':<15} {'ΔH‡ (kJ/mol)':<15} {'ΔS‡ (J/K·mol)':<18} {'Ea (kJ/mol)':<15} {'R²':<10}")
    print("-"*80)
    
    for candidate, params in results.items():
        print(f"{candidate:<15} {params['dH']:>12.1f}   {params['dS']:>15.1f}   {params['Ea']:>12.1f}   {params['r2']:>8.4f}")
    
    print("\n" + "="*80)
    print("ANALYSIS QUESTIONS")
    print("="*80)
    
    print("\n1. Which candidate has the highest activation barrier (ΔH‡)?")
    print("   This candidate will be the SLOWEST at body temperature.")
    
    print("\n2. Compare ΔS‡ values. What do negative values tell you?")
    print("   Negative ΔS‡ → Ordered transition state (associative mechanism)")
    print("   Positive ΔS‡ → Disordered transition state (dissociative mechanism)")
    
    print("\n3. Note that Ea ≈ ΔH‡ + RT (at ~310 K, RT ≈ 2.6 kJ/mol)")
    print("   This is the relationship between Arrhenius and Eyring formalisms!")
    
    print("\n" + "="*80)

### Understanding Activation Parameters

**Enthalpy of Activation (ΔH‡)**:
- Measures the **energy barrier** height
- Higher ΔH‡ → Slower reaction
- Related to bond breaking/forming in transition state

**Entropy of Activation (ΔS‡)**:
- Measures the **order/disorder** change
- Negative ΔS‡ → Transition state is more rigid/ordered ("order tax")
- Positive ΔS‡ → Transition state is more flexible/disordered

**Design Implications**:
- To slow down a reaction: Increase ΔH‡ or make ΔS‡ more negative
- To speed up a reaction: Decrease ΔH‡ or make ΔS‡ more positive

---

## INVESTIGATION 2: Mechanism Elucidation via KIE 🔬

### YOUR TASK
Determine if C-H bond breaking is involved in the rate-limiting step by measuring the Kinetic Isotope Effect.

### EXERCISE 2.1: Kinetic Isotope Effect Analysis

## 3. Kinetic Isotope Effect (KIE) and Tunneling

Replacing a hydrogen atom (H) with deuterium (D) often slows down a reaction. Why?

### Zero-Point Energy (ZPE)
Chemical bonds vibrate even at absolute zero. The energy is $E_0 = \frac{1}{2} h \nu$.
Since $\nu \propto 1/\sqrt{\mu}$, and $\mu_D > \mu_H$, the C-D bond has lower frequency and lower ZPE than C-H.

$$ ZPE_{C-D} < ZPE_{C-H} $$

This means the C-H bond starts from a higher energy "platform" than C-D, so it needs less activation energy to reach the top.

$$ k_H > k_D $$

The maximum primary KIE ($k_H/k_D$) at room temperature is about 7.

### Quantum Tunneling
Light particles like H (and especially $e^-$) can tunnel *through* the barrier instead of going over it. Tunneling is much harder for D (heavier).

**Evidence for Tunneling:**
1.  $k_H/k_D \gg 7$ (can be > 100!).
2.  Curved Arrhenius plots (concave up) at low temperatures.

In [ ]:
# EXERCISE 2.1: Kinetic Isotope Effect Analysis

# Load KIE data
try:
    kie_data = pd.read_csv('data/tst/prodrug_kie_data.csv')
    print("✓ Kinetic Isotope Effect data loaded!")
    print(f"\nData shape: {kie_data.shape}")
    print(f"\nData:")
    print(kie_data)
except FileNotFoundError:
    print("⚠️ Data file not found.")
    kie_data = None

if kie_data is not None:
    # Calculate KIE for each candidate
    candidates = kie_data['candidate'].unique()
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    print("\n" + "="*80)
    print("KINETIC ISOTOPE EFFECT ANALYSIS")
    print("="*80)
    print(f"\n{'Candidate':<15} {'kH (s⁻¹)':<15} {'kD (s⁻¹)':<15} {'KIE (kH/kD)':<15}")
    print("-"*80)
    
    kie_values = []
    
    for candidate in candidates:
        data = kie_data[kie_data['candidate'] == candidate]
        kH = data[data['isotope'] == 'H']['rate_constant_s_inv'].values[0]
        kD = data[data['isotope'] == 'D']['rate_constant_s_inv'].values[0]
        kie = kH / kD
        kie_values.append(kie)
        
        print(f"{candidate:<15} {kH:>12.2e}   {kD:>12.2e}   {kie:>12.2f}")
    
    # Plot 1: Bar chart of KIE values
    x_pos = np.arange(len(candidates))
    axes[0].bar(x_pos, kie_values, alpha=0.7, color=['blue', 'green', 'red'],
               edgecolor='black', linewidth=2)
    axes[0].axhline(7, color='orange', linestyle='--', linewidth=2, 
                   label='Classical maximum (KIE ≈ 7)')
    axes[0].axhline(1, color='gray', linestyle=':', linewidth=1)
    axes[0].set_xticks(x_pos)
    axes[0].set_xticklabels(candidates)
    axes[0].set_ylabel('KIE (kH/kD)', fontsize=12)
    axes[0].set_title('Kinetic Isotope Effects', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Plot 2: Temperature dependence of KIE for Candidate B
    # (to check for tunneling)
    T_range = np.linspace(280, 330, 6)
    
    # Simulate temperature-dependent KIE
    # Classical: KIE ~ exp(Δ(ZPE)/kT)
    # Assuming C-H stretch at 3000 cm^-1
    wavenumber = 2900  # cm^-1
    c_cm = constants.c * 100  # cm/s
    nu_H = wavenumber * c_cm
    nu_D = nu_H / np.sqrt(2)
    delta_ZPE = 0.5 * h * (nu_H - nu_D)
    
    kie_classical = np.exp(delta_ZPE / (k_B * T_range))
    
    axes[1].plot(T_range, kie_classical, 'b-', linewidth=2, marker='o',
                label='Classical prediction')
    axes[1].axhline(7, color='orange', linestyle='--', linewidth=1,
                   label='Classical max at 298 K')
    axes[1].set_xlabel('Temperature (K)', fontsize=12)
    axes[1].set_ylabel('KIE (kH/kD)', fontsize=12)
    axes[1].set_title('Temperature Dependence of KIE', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*80)
    print("MECHANISTIC INTERPRETATION")
    print("="*80)
    
    for i, candidate in enumerate(candidates):
        print(f"\n{candidate}:")
        if kie_values[i] < 2:
            print("  → Small KIE: C-H bond breaking NOT in rate-limiting step")
            print("  → Likely diffusion-controlled or non-hydride mechanism")
        elif 2 <= kie_values[i] <= 7:
            print("  → Moderate KIE: C-H bond breaking IS rate-limiting")
            print("  → Classical transition state (no tunneling)")
        else:
            print("  → Large KIE (>7): Strong evidence for QUANTUM TUNNELING!")
            print("  → Hydrogen can tunnel through barrier")
    
    print("\n💡 DESIGN INSIGHT:")
    print("   Replacing H with D (deuteration) can stabilize prodrugs by increasing t½!")
    print("   This is used in real drug development (e.g., deutetrabenazine).")
    print("="*80)

---

## INVESTIGATION 3: Structure-Activity Optimization via Hammett Analysis 🎯

### YOUR TASK
Use Hammett plots to predict how different substituents on the aromatic ring will affect prodrug activation rate.

### EXERCISE 3.1: Hammett Plot Analysis

## 4. Linear Free Energy Relationships (LFER)

How does changing the structure of a molecule affect its reactivity? The **Hammett Equation** relates reaction rates to equilibrium constants for substituted benzenes:

$$ \log \left( \frac{k}{k_H} \right) = \rho \sigma $$

-   **$\sigma$ (Substituent Constant)**: Measures the electronic effect of the substituent (positive = electron withdrawing, negative = electron donating).
-   **$\rho$ (Reaction Constant)**: Measures the sensitivity of the reaction to electronic effects.
    -   $\rho > 0$: Reaction is accelerated by electron withdrawal (negative charge buildup in transition state).
    -   $\rho < 0$: Reaction is accelerated by electron donation (positive charge buildup in transition state).

In [ ]:
# EXERCISE 3.1: Hammett Analysis for Prodrug Optimization

# Load Hammett data
try:
    hammett_data = pd.read_csv('data/tst/prodrug_hammett_data.csv')
    print("✓ Hammett analysis data loaded!")
    print(f"\nData shape: {hammett_data.shape}")
    print(f"\nData:")
    print(hammett_data)
except FileNotFoundError:
    print("⚠️ Data file not found.")
    hammett_data = None

if hammett_data is not None:
    import scipy.stats as stats
    
    # Calculate log(k/kH)
    k_H = hammett_data[hammett_data['substituent'] == 'H']['rate_constant_s_inv'].values[0]
    hammett_data['log_k_rel'] = np.log10(hammett_data['rate_constant_s_inv'] / k_H)
    
    # Linear regression
    sigma = hammett_data['sigma_para'].values
    log_k_rel = hammett_data['log_k_rel'].values
    
    rho, intercept, r_value, p_value, std_err = stats.linregress(sigma, log_k_rel)
    
    # Plotting
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot 1: Hammett plot
    axes[0].scatter(sigma, log_k_rel, s=150, alpha=0.7, c='purple',
                   edgecolors='black', linewidth=2)
    axes[0].plot(sigma, rho*sigma + intercept, 'k--', linewidth=2,
                label=f'ρ = {rho:.2f} (R² = {r_value**2:.3f})')
    
    # Annotate points
    for _, row in hammett_data.iterrows():
        axes[0].annotate(row['substituent'], 
                        (row['sigma_para'], row['log_k_rel']),
                        xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    axes[0].axhline(0, color='gray', linestyle=':', linewidth=1)
    axes[0].axvline(0, color='gray', linestyle=':', linewidth=1)
    axes[0].set_xlabel('Hammett σ (para)', fontsize=12)
    axes[0].set_ylabel('log(k / kH)', fontsize=12)
    axes[0].set_title('Hammett Plot: Structure-Reactivity', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Predicted vs Observed
    predicted = rho * sigma + intercept
    axes[1].scatter(predicted, log_k_rel, s=150, alpha=0.7, c='green',
                   edgecolors='black', linewidth=2)
    # Diagonal line
    lim = max(abs(predicted.min()), abs(predicted.max()), 
              abs(log_k_rel.min()), abs(log_k_rel.max()))
    axes[1].plot([-lim, lim], [-lim, lim], 'r--', linewidth=2, label='Perfect fit')
    axes[1].set_xlabel('Predicted log(k/kH)', fontsize=12)
    axes[1].set_ylabel('Observed log(k/kH)', fontsize=12)
    axes[1].set_title('Model Quality', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Analysis
    print("\n" + "="*80)
    print("HAMMETT ANALYSIS RESULTS")
    print("="*80)
    print(f"\nReaction constant ρ = {rho:.3f}")
    print(f"R² = {r_value**2:.4f}")
    print(f"p-value = {p_value:.2e}")
    
    print("\n📊 MECHANISTIC INTERPRETATION:")
    if rho > 0.5:
        print(f"  ρ > 0 → Electron-withdrawing groups ACCELERATE the reaction")
        print(f"  → Negative charge develops in the transition state")
        print(f"  → Likely nucleophilic attack mechanism")
    elif rho < -0.5:
        print(f"  ρ < 0 → Electron-donating groups ACCELERATE the reaction")
        print(f"  → Positive charge develops in the transition state")
        print(f"  → Likely electrophilic attack or carbocation mechanism")
    else:
        print(f"  ρ ≈ 0 → Little charge development in transition state")
        print(f"  → Radical or concerted mechanism")
    
    print("\n💡 DESIGN RECOMMENDATIONS:")
    if rho > 0:
        print("  To SPEED UP: Add electron-withdrawing groups (Cl, CN, NO2)")
        print("  To SLOW DOWN: Add electron-donating groups (NH2, OCH3, CH3)")
    else:
        print("  To SPEED UP: Add electron-donating groups (NH2, OCH3, CH3)")
        print("  To SLOW DOWN: Add electron-withdrawing groups (Cl, CN, NO2)")
    
    # Predict best substituents
    print("\n🎯 OPTIMIZATION TARGETS:")
    
    # Target half-life: 2-4 hours = rate constant ~ 1e-4 s^-1
    target_k = 1e-4  # s^-1
    target_log_k_rel = np.log10(target_k / k_H)
    target_sigma = (target_log_k_rel - intercept) / rho
    
    print(f"  Target half-life: 2-4 hours")
    print(f"  Target rate constant: ~{target_k:.2e} s⁻¹")
    print(f"  Required σ: {target_sigma:.2f}")
    
    # Find closest substituent
    hammett_data['sigma_diff'] = np.abs(hammett_data['sigma_para'] - target_sigma)
    best = hammett_data.loc[hammett_data['sigma_diff'].idxmin()]
    print(f"\n  📌 BEST CANDIDATE: {best['substituent']}")
    print(f"     σ = {best['sigma_para']:.2f}")
    print(f"     Predicted k = {best['rate_constant_s_inv']:.2e} s⁻¹")
    print(f"     Predicted t½ = {np.log(2)/best['rate_constant_s_inv']/3600:.2f} hours")
    
    print("\n" + "="*80)

---

## INVESTIGATION 4: Potential Energy Surfaces and Transition States 🗻

### YOUR TASK
Construct a LEPS potential energy surface and locate the transition state using Newton-Raphson optimization.

In this investigation, you'll move from the thermodynamic view of TST to the actual *potential energy surface* (PES) that governs a chemical reaction. You'll:

1. Construct a LEPS (London-Eyring-Polanyi-Sato) surface for H + HI → HI + H
2. Visualize the energy landscape in 3D and as contour plots
3. Find the exact transition state geometry using Newton-Raphson optimization
4. Calculate force constants and vibrational frequencies at the saddle point

This connects theory to computation - exactly how modern quantum chemistry finds transition states!

### EXERCISE 4.1: Constructing the LEPS Potential Energy Surface

In [ ]:
# EXERCISE 4.1: LEPS Potential Energy Surface# Import the LEPS moduleimport syssys.path.append('../modules')from leps_surface import LEPSSurfacefrom visualization import plot_pes_3d, plot_pes_contour, plot_morse_curveprint("="*80)print("CONSTRUCTING LEPS POTENTIAL ENERGY SURFACE")print("="*80)# Create LEPS surface for H + HI reactionsurface = LEPSSurface('HI', 'HI', 'I2', K_sato=0.0)print("\n✓ LEPS surface initialized for H + H-I system")print("\nMorse parameters loaded:")print(f"  H-I bond: D_e = {surface.params['AB']['D_e']:.2f} kJ/mol, " +      f"R_e = {surface.params['AB']['R_e']:.3f} Å")# Visualize Morse potentialR_range = np.linspace(0.8, 5.0, 200)params_HI = surface.params['AB']V_morse = surface.morse_potential(R_range, params_HI['D_e'], params_HI['R_e'], params_HI['beta'])fig, ax = plot_morse_curve(R_range, V_morse, molecule_name="HI",                           D_e=params_HI['D_e'], R_e=params_HI['R_e'])plt.show()print(f"\n✓ Morse potential minimum at R_e = {params_HI['R_e']:.3f} Å")

In [ ]:
# EXERCISE 4.2: 2D Potential Energy Surfaceprint("\n" + "="*80)print("GENERATING 2D POTENTIAL ENERGY SURFACE")print("="*80)# Generate 2D surfaceR_AB_range = np.linspace(1.0, 4.5, 60)R_BC_range = np.linspace(1.0, 4.5, 60)print("\nCalculating LEPS potential on 60x60 grid...")R_AB_grid, R_BC_grid, V_grid = surface.energy_surface_2d(R_AB_range, R_BC_range, angle_deg=180.0)print(f"✓ Surface calculated! Grid shape: {V_grid.shape}")print(f"  Energy range: {V_grid.min():.1f} to {V_grid.max():.1f} kJ/mol")# Create 3D visualizationfig_3d, ax_3d = plot_pes_3d(R_AB_grid, R_BC_grid, V_grid,                            title="LEPS Surface: H + HI -> HI + H",                            xlabel="R(H···H) (Å)", ylabel="R(H-I) (Å)")plt.show()# Create contour plotfig_contour, ax_contour = plot_pes_contour(R_AB_grid, R_BC_grid, V_grid,                                            title="LEPS Energy Contours",                                            xlabel="R(H···H) (Å)", ylabel="R(H-I) (Å)",                                            levels=40, vmin=-320, vmax=-180)plt.show()print("\n📊 The saddle point (transition state) is visible in the middle of the contour map")

In [ ]:
# EXERCISE 4.3: Newton-Raphson Transition State Optimizationfrom transition_state import TransitionStateOptimizerprint("\n" + "="*80)print("TRANSITION STATE OPTIMIZATION")print("="*80)# Create optimizeroptimizer = TransitionStateOptimizer(surface, tolerance=1e-6, max_iterations=50)# Initial guessR_AB_init, R_BC_init = 1.9, 1.9print(f"\nInitial guess: R_AB = {R_AB_init:.2f} Å, R_BC = {R_BC_init:.2f} Å\n")# Run optimizationresult = optimizer.optimize_saddle_point(R_AB_init, R_BC_init, verbose=True)history_df = result['history']# Check saddle pointeigenvalues = result['eigenvalues']print(f"\nHessian eigenvalues: {eigenvalues}")if eigenvalues[0] < 0 and eigenvalues[1] > 0:    print("✓ Confirmed saddle point (one negative eigenvalue)")# Calculate activation energyE_reactants = surface.leps_potential(3.0, surface.params['BC']['R_e'],                                     3.0 + surface.params['BC']['R_e'])Ea = result['energy'] - E_reactantsprint(f"\n📊 Activation energy: {Ea:.2f} kJ/mol")

In [ ]:
# EXERCISE 4.4: Visualization of Optimization Convergencefig, axes = plt.subplots(1, 2, figsize=(14, 6))# Energy convergenceaxes[0].plot(history_df['iteration'], history_df['energy'], 'b-o', linewidth=2)axes[0].set_xlabel('Iteration')axes[0].set_ylabel('Energy (kJ/mol)')axes[0].set_title('Energy Convergence', fontweight='bold')axes[0].grid(True, alpha=0.3)# Optimization path on contouraxes[1].contourf(R_AB_grid, R_BC_grid, V_grid, levels=40, cmap='viridis',                 vmin=-320, vmax=-180, alpha=0.6)axes[1].plot(history_df['R_AB'], history_df['R_BC'], 'r-o', linewidth=3,             label='Optimization path', markeredgecolor='white', markeredgewidth=2)axes[1].plot(history_df['R_AB'].iloc[0], history_df['R_BC'].iloc[0],             'go', markersize=15, label='Start')axes[1].plot(history_df['R_AB'].iloc[-1], history_df['R_BC'].iloc[-1],             'r*', markersize=20, label='Saddle point')axes[1].set_xlabel('R(H···H) (Å)')axes[1].set_ylabel('R(H-I) (Å)')axes[1].set_title('Optimization Path on PES', fontweight='bold')axes[1].legend()plt.tight_layout()plt.show()print(f"\n✓ Converged in {result['iterations']} iterations")print(f"  Final: R_AB = {result['R_AB']:.6f} Å, R_BC = {result['R_BC']:.6f} Å")

In [ ]:
# EXERCISE 4.1: LEPS Potential Energy Surface# Import the LEPS moduleimport syssys.path.append('../modules')from leps_surface import LEPSSurfacefrom visualization import plot_pes_3d, plot_pes_contour, plot_morse_curveprint("="*80)print("CONSTRUCTING LEPS POTENTIAL ENERGY SURFACE")print("="*80)# Create LEPS surface for H + HI reactionsurface = LEPSSurface('HI', 'HI', 'I2', K_sato=0.0)print("\n✓ LEPS surface initialized for H + H-I system")print("\nMorse parameters loaded:")print(f"  H-I bond: D_e = {surface.params['AB']['D_e']:.2f} kJ/mol, " +      f"R_e = {surface.params['AB']['R_e']:.3f} Å")# Visualize Morse potentialR_range = np.linspace(0.8, 5.0, 200)params_HI = surface.params['AB']V_morse = surface.morse_potential(R_range, params_HI['D_e'], params_HI['R_e'], params_HI['beta'])fig, ax = plot_morse_curve(R_range, V_morse, molecule_name="HI",                           D_e=params_HI['D_e'], R_e=params_HI['R_e'])plt.show()print(f"\n✓ Morse potential minimum at R_e = {params_HI['R_e']:.3f} Å")

In [ ]:
# EXERCISE 4.2: 2D Potential Energy Surfaceprint("\n" + "="*80)print("GENERATING 2D POTENTIAL ENERGY SURFACE")print("="*80)# Generate 2D surfaceR_AB_range = np.linspace(1.0, 4.5, 60)R_BC_range = np.linspace(1.0, 4.5, 60)print("\nCalculating LEPS potential on 60x60 grid...")R_AB_grid, R_BC_grid, V_grid = surface.energy_surface_2d(R_AB_range, R_BC_range, angle_deg=180.0)print(f"✓ Surface calculated! Grid shape: {V_grid.shape}")print(f"  Energy range: {V_grid.min():.1f} to {V_grid.max():.1f} kJ/mol")# Create 3D visualizationfig_3d, ax_3d = plot_pes_3d(R_AB_grid, R_BC_grid, V_grid,                            title="LEPS Surface: H + HI -> HI + H",                            xlabel="R(H···H) (Å)", ylabel="R(H-I) (Å)")plt.show()# Create contour plotfig_contour, ax_contour = plot_pes_contour(R_AB_grid, R_BC_grid, V_grid,                                            title="LEPS Energy Contours",                                            xlabel="R(H···H) (Å)", ylabel="R(H-I) (Å)",                                            levels=40, vmin=-320, vmax=-180)plt.show()print("\n📊 The saddle point (transition state) is visible in the middle of the contour map")

In [ ]:
# EXERCISE 4.3: Newton-Raphson Transition State Optimizationfrom transition_state import TransitionStateOptimizerprint("\n" + "="*80)print("TRANSITION STATE OPTIMIZATION")print("="*80)# Create optimizeroptimizer = TransitionStateOptimizer(surface, tolerance=1e-6, max_iterations=50)# Initial guessR_AB_init, R_BC_init = 1.9, 1.9print(f"\nInitial guess: R_AB = {R_AB_init:.2f} Å, R_BC = {R_BC_init:.2f} Å\n")# Run optimizationresult = optimizer.optimize_saddle_point(R_AB_init, R_BC_init, verbose=True)history_df = result['history']# Check saddle pointeigenvalues = result['eigenvalues']print(f"\nHessian eigenvalues: {eigenvalues}")if eigenvalues[0] < 0 and eigenvalues[1] > 0:    print("✓ Confirmed saddle point (one negative eigenvalue)")# Calculate activation energyE_reactants = surface.leps_potential(3.0, surface.params['BC']['R_e'],                                     3.0 + surface.params['BC']['R_e'])Ea = result['energy'] - E_reactantsprint(f"\n📊 Activation energy: {Ea:.2f} kJ/mol")

In [ ]:
# EXERCISE 4.4: Visualization of Optimization Convergencefig, axes = plt.subplots(1, 2, figsize=(14, 6))# Energy convergenceaxes[0].plot(history_df['iteration'], history_df['energy'], 'b-o', linewidth=2)axes[0].set_xlabel('Iteration')axes[0].set_ylabel('Energy (kJ/mol)')axes[0].set_title('Energy Convergence', fontweight='bold')axes[0].grid(True, alpha=0.3)# Optimization path on contouraxes[1].contourf(R_AB_grid, R_BC_grid, V_grid, levels=40, cmap='viridis',                 vmin=-320, vmax=-180, alpha=0.6)axes[1].plot(history_df['R_AB'], history_df['R_BC'], 'r-o', linewidth=3,             label='Optimization path', markeredgecolor='white', markeredgewidth=2)axes[1].plot(history_df['R_AB'].iloc[0], history_df['R_BC'].iloc[0],             'go', markersize=15, label='Start')axes[1].plot(history_df['R_AB'].iloc[-1], history_df['R_BC'].iloc[-1],             'r*', markersize=20, label='Saddle point')axes[1].set_xlabel('R(H···H) (Å)')axes[1].set_ylabel('R(H-I) (Å)')axes[1].set_title('Optimization Path on PES', fontweight='bold')axes[1].legend()plt.tight_layout()plt.show()print(f"\n✓ Converged in {result['iterations']} iterations")print(f"  Final: R_AB = {result['R_AB']:.6f} Å, R_BC = {result['R_BC']:.6f} Å")

---

## PHASE 3: SYNTHESIZE 🎯

## CAPSTONE CHALLENGE: Final Prodrug Recommendation

Time to integrate all your analyses and deliver your recommendation to the clinical team!

---

### THE COMPLETE PICTURE

You now have:
1. ✅ Activation parameters (ΔH‡, ΔS‡) for three candidates
2. ✅ Mechanistic insights from KIE
3. ✅ Structure-activity relationships from Hammett analysis

Your task is to:
1. **Synthesize** all data to select the best prodrug candidate
2. **Propose** chemical modifications if needed
3. **Predict** performance at physiological conditions (37°C, pH 7.4)
4. **Write** a technical recommendation memo

Let's make the final assessment:

In [ ]:
# CAPSTONE CHALLENGE: Integrated Analysis and Recommendation

print("="*80)
print("FINAL PRODRUG EVALUATION: INTEGRATED ANALYSIS")
print("="*80)

# Combine all data
if kinetics_data is not None and kie_data is not None:
    
    # Calculate half-lives at 37°C (310 K)
    T_body = 310  # K
    
    print("\n📋 CANDIDATE SUMMARY AT BODY TEMPERATURE (37°C)")
    print("-"*80)
    print(f"\n{'Candidate':<12} {'ΔH‡':<10} {'ΔS‡':<12} {'KIE':<8} {'k (s⁻¹)':<12} {'t½ (hours)':<12}")
    print("-"*80)
    
    recommendations = []
    
    for candidate in ['Candidate A', 'Candidate B', 'Candidate C']:
        # Get activation parameters
        if candidate in results:
            dH = results[candidate]['dH'] * 1000  # J/mol
            dS = results[candidate]['dS']  # J/(K·mol)
            
            # Calculate k at body temp using Eyring equation
            k_body = (k_B * T_body / h) * np.exp(dS/R) * np.exp(-dH/(R*T_body))
            
            # Half-life
            t_half_hours = np.log(2) / k_body / 3600
            
            # Get KIE
            kie_val = kie_values[['Candidate A', 'Candidate B', 'Candidate C'].index(candidate)]
            
            print(f"{candidate:<12} {results[candidate]['dH']:>7.1f} kJ {dS:>8.1f} J/K {kie_val:>6.2f}  {k_body:>10.2e}  {t_half_hours:>10.2f}")
            
            # Evaluation
            score = 0
            comments = []
            
            # Target: t½ = 2-4 hours
            if 2 <= t_half_hours <= 4:
                score += 50
                comments.append("✓ OPTIMAL half-life (2-4 hrs)")
            elif 1 <= t_half_hours <= 6:
                score += 30
                comments.append("~ Acceptable half-life (1-6 hrs)")
            else:
                score += 10
                if t_half_hours < 1:
                    comments.append("✗ Too fast - toxicity risk")
                else:
                    comments.append("✗ Too slow - no efficacy")
            
            # Moderate KIE preferred (predictable, no tunneling)
            if 3 <= kie_val <= 7:
                score += 20
                comments.append("✓ Classical KIE (predictable)")
            else:
                score += 10
                comments.append("~ Non-standard KIE")
            
            # Moderate ΔH‡ (tunable)
            if 60 <= results[candidate]['dH'] <= 90:
                score += 15
                comments.append("✓ Tunable barrier height")
            else:
                score += 5
            
            # Not too negative ΔS‡ (manufacturability)
            if dS > -100:
                score += 15
                comments.append("✓ Favorable entropy")
            else:
                score += 5
                comments.append("~ Large entropy penalty")
            
            recommendations.append({
                'candidate': candidate,
                'score': score,
                'comments': comments,
                't_half': t_half_hours
            })
    
    print("\n" + "="*80)
    print("DETAILED EVALUATION")
    print("="*80)
    
    # Sort by score
    recommendations.sort(key=lambda x: x['score'], reverse=True)
    
    for i, rec in enumerate(recommendations):
        print(f"\n{'🥇' if i==0 else '🥈' if i==1 else '🥉'} {rec['candidate']} - Score: {rec['score']}/100")
        for comment in rec['comments']:
            print(f"   {comment}")
    
    # Final recommendation
    best = recommendations[0]
    
    print("\n" + "="*80)
    print("🎯 FINAL RECOMMENDATION")
    print("="*80)
    print(f"\n**RECOMMENDED FOR CLINICAL TRIALS: {best['candidate']}**")
    print(f"\nPredicted half-life at 37°C: {best['t_half']:.2f} hours")
    print(f"Overall score: {best['score']}/100")
    
    print("\n📝 RATIONALE:")
    for comment in best['comments']:
        print(f"  • {comment}")
    
    print("\n💊 ADDITIONAL OPTIMIZATION STRATEGIES:")
    if best['t_half'] < 2:
        print("  • Consider deuteration to slow down activation (increase t½)")
        print("  • Add electron-donating groups if ρ > 0")
        print("  • Increase steric bulk near reactive center")
    elif best['t_half'] > 4:
        print("  • Add electron-withdrawing groups if ρ > 0")
        print("  • Reduce steric hindrance")
        print("  • Consider alternative leaving groups")
    else:
        print("  • Current structure is optimal - proceed to formulation studies")
        print("  • Monitor stability across pH range 6.5-7.8")
        print("  • Validate in vivo pharmacokinetics")
    
    print("\n" + "="*80)

### 📝 YOUR TECHNICAL MEMO

Based on your comprehensive analysis, write a brief recommendation to the clinical development team:

**MEMO TEMPLATE:**

**TO**: Clinical Development Team  
**FROM**: [Your Name], Medicinal Chemistry  
**RE**: Prodrug Candidate Recommendation for Phase I Trials

**EXECUTIVE SUMMARY**:
1. Which candidate do you recommend and why?
2. What is the predicted half-life and how does it meet clinical requirements?
3. What mechanistic insights support this choice?
4. What are the potential risks or limitations?
5. What additional studies are recommended before clinical trials?

---

### 🎓 LEARNING OBJECTIVES - REVIEW

Go back and check off your completed objectives:
- [x] Apply the Eyring equation to calculate rate constants from activation energies
- [x] Extract ΔH‡ and ΔS‡ from temperature-dependent kinetics data
- [x] Use Kinetic Isotope Effects to determine reaction mechanisms
- [x] Apply Hammett plots to predict structure-reactivity relationships
- [x] Optimize reaction conditions (pH, ionic strength) using TST principles
- [x] Design molecules with targeted activation barriers

**Excellent work! You've mastered transition-state theory!**

---

### 🤔 FINAL REFLECTION

1. **Real-World Application**: How is TST used in modern drug discovery?
   - Computational chemistry uses TST to predict reaction rates
   - Deuteration is a real strategy (FDA-approved drugs: deutetrabenazine, etc.)
   - Hammett analysis guides medicinal chemistry optimization

2. **Limitations**: When does TST fail?
   - Very fast reactions (diffusion-controlled)
   - Non-equilibrium conditions
   - Reactions with recrossing (transmission coefficient κ << 1)

3. **Connection to Next Topics**: How does TST relate to molecular dynamics (Notebook 04)?
   - MD simulations can compute TST parameters from first principles!
   - Can observe individual molecules crossing the barrier
   - Can calculate transmission coefficients

---

### 📚 EXTENSIONS

Want to go deeper?
- Variational Transition State Theory (VTST)
- Marcus Theory for electron transfer (Notebook 05)
- Enzymatic catalysis and TST
- Computational prediction of activation barriers (DFT)

---

## CONGRATULATIONS! 🎉

You've successfully:
- ✅ Applied Eyring equation to extract activation parameters
- ✅ Used KIE to elucidate reaction mechanisms
- ✅ Employed Hammett analysis for molecular design
- ✅ Optimized a prodrug using transition-state theory

**Ready for Notebook 04: Molecular Dynamics!**